In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install tensorboardX
! pip install tensorrt

In [ ]:
!pip install cython -U
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --quiet --editable .
!pip install --quiet sentencepiece

# Baseline

Result:

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    ! CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/mnli/fiction/orig/bin \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/fiction/baseline \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'mnli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-09 10:20:35.944380: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 10:20:37 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-09 10:20:44 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/fiction/baseline',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/mnli/fiction/orig/bin'
)
roberta.eval()

In [ ]:
genre = ['slate','fiction','telephone','travel','government']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1


  print(g,": ",accuracy)
  print(g,": ",total)

slate :  710
slate :  2000
fiction :  734
fiction :  2000
telephone :  688
telephone :  2000
travel :  704
travel :  2000
government :  741
government :  2000


In [ ]:
genre = ['verbatim','facetoface','oup','nineeleven','letters']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

verbatim :  359
verbatim :  1000
facetoface :  369
facetoface :  1000
oup :  360
oup :  1000
nineeleven :  365
nineeleven :  1000
letters :  367
letters :  1000


# EDA

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    ! CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/mnli/fiction/orig/bin_eda \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/fiction/eda \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'mnli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-09 10:31:17.244614: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 10:31:20 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-09 10:31:24 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/fiction/eda',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/mnli/fiction/orig/bin_eda'
)
roberta.eval()

In [ ]:
genre = ['slate','fiction','telephone','travel','government']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

slate :  768
slate :  2000
fiction :  793
fiction :  2000
telephone :  801
telephone :  2000
travel :  750
travel :  2000
government :  747
government :  2000


In [ ]:
genre = ['verbatim','facetoface','oup','nineeleven','letters']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

verbatim :  416
verbatim :  1000
facetoface :  385
facetoface :  1000
oup :  396
oup :  1000
nineeleven :  382
nineeleven :  1000
letters :  374
letters :  1000


# UDA

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    ! CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/mnli/fiction/orig/bin_uda \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/fiction/uda \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'mnli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-09 11:26:37.001679: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 11:26:38 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-09 11:26:41 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/fiction/uda',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/mnli/fiction/orig/bin_uda'
)
roberta.eval()

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-11): 12 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [ ]:
genre = ['slate','fiction','telephone','travel','government']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1


  print(g,": ",accuracy)
  print(g,": ",total)

slate :  785
slate :  2000
fiction :  867
fiction :  2000
telephone :  829
telephone :  2000
travel :  793
travel :  2000
government :  821
government :  2000


In [ ]:
genre = ['verbatim','facetoface','oup','nineeleven','letters']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

verbatim :  428
verbatim :  1000
facetoface :  424
facetoface :  1000
oup :  399
oup :  1000
nineeleven :  420
nineeleven :  1000
letters :  413
letters :  1000


# SSMBA

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    ! CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/mnli/fiction/orig/bin_ssmba2 \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/fiction/ssmba \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'mnli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-09 15:08:32.890071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 15:08:34 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-09 15:08:39 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/fiction/ssmba',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/mnli/fiction/orig/bin_ssmba'
)
roberta.eval()

In [ ]:
genre = ['slate','fiction','telephone','travel','government']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

slate :  766
slate :  2000
fiction :  821
fiction :  2000
telephone :  799
telephone :  2000
travel :  737
travel :  2000
government :  749
government :  2000


In [ ]:
genre = ['verbatim','facetoface','oup','nineeleven','letters']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

verbatim :  415
verbatim :  1000
facetoface :  388
facetoface :  1000
oup :  398
oup :  1000
nineeleven :  381
nineeleven :  1000
letters :  382
letters :  1000


# EDA Extended

In [ ]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
    ! CUDA_VISIBLE_DEVICES=0 fairseq-train /content/drive/MyDrive/NLP/mnli/fiction/orig/bin_eda_extended \
          --save-dir /content/drive/MyDrive/NLP/checkpoints/fiction/eda_extended \
          --reset-optimizer  \
          --reset-dataloader  \
          --reset-meters  \
          --best-checkpoint-metric accuracy  \
          --maximize-best-checkpoint-metric  \
          --no-epoch-checkpoints \
          --find-unused-parameters \
          --distributed-world-size 1 \
          --task sentence_prediction  \
          --num-classes 3  \
          --init-token 0  \
          --separator-token 2   \
          --max-positions 512  \
          --shorten-method "truncate"  \
          --arch roberta \
          --dropout 0.1  \
          --attention-dropout 0.1  \
          --weight-decay 0.1  \
          --criterion sentence_prediction  \
          --classification-head-name 'mnli' \
          --optimizer adam  \
          --adam-betas '(0.9, 0.98)'  \
          --adam-eps 1e-06  \
          --clip-norm 0.0  \
          --lr-scheduler inverse_sqrt  \
          --lr 1e-05 \
          --fp16  \
          --fp16-init-scale 4  \
          --threshold-loss-scale 1  \
          --fp16-scale-window 128  \
          --batch-size 32  \
          --required-batch-size-multiple 1  \
          --max-tokens 4400 \
          --update-freq 1  \
          --max-update 123873 \
          --max-epoch 5 \
          --seed 100

2023-06-15 09:55:41.902053: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-15 09:55:43 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-15 09:55:48 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 100, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parall

In [ ]:
from fairseq.models.roberta import RobertaModel
roberta = RobertaModel.from_pretrained(
    '/content/drive/MyDrive/NLP/checkpoints/fiction/eda_extended',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/content/drive/MyDrive/NLP/mnli/fiction/orig/bin_eda_extended'
)
roberta.eval()

In [ ]:
genre = ['slate','fiction','telephone','travel','government','verbatim','facetoface','oup','nineeleven','letters']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

slate :  730
slate :  2000
fiction :  782
fiction :  2000
telephone :  747
telephone :  2000
travel :  735
travel :  2000
government :  703
government :  2000
verbatim :  402
verbatim :  1000
facetoface :  389
facetoface :  1000
oup :  383
oup :  1000
nineeleven :  389
nineeleven :  1000
letters :  357
letters :  1000


In [ ]:
genre = ['slate','fiction','telephone','travel','government','verbatim','facetoface','oup','nineeleven','letters']
for g in genre:
  input0 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input0', "r")
  input1 = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.input1', "r")
  label = open('/content/drive/MyDrive/NLP/mnli/'+g+'/orig/test.raw.label', "r")

  accuracy = 0
  total = 0
  for (x1, x2, y) in zip(input0, input1, label):
    tokens = roberta.encode(x1, x2)
    idx = roberta.predict('mnli', tokens).argmax().item()
    dictionary = roberta.task.label_dictionary
    pred = dictionary[idx + dictionary.nspecial]
    total = total + 1
    if  (pred == y.strip()) :
      accuracy = accuracy + 1

  print(g,": ",accuracy)
  print(g,": ",total)

slate :  766
slate :  2000
fiction :  758
fiction :  2000
telephone :  793
telephone :  2000
travel :  781
travel :  2000
government :  733
government :  2000
verbatim :  401
verbatim :  1000
facetoface :  378
facetoface :  1000
oup :  403
oup :  1000
nineeleven :  373
nineeleven :  1000
letters :  382
letters :  1000
